In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from multiprocessing import Pool

os.chdir("../../")

In [7]:
i = 0
class data_preprocessing:
    
    paths = []
    
    def __init__(self):
        with open("Data/hierarchy_data.pkl","rb") as f:
            hd = pkl.load(f)
        with open("Data/class_encoding.pkl","rb") as f:
            self.classes = pkl.load(f)
        with open("Data/order_encoding.pkl","rb") as f:
            self.orders = pkl.load(f)
        with open("Data/family_encoding.pkl","rb") as f:
            self.families = pkl.load(f)
        with open("Data/genus_encoding.pkl","rb") as f:
            self.genuses = pkl.load(f)
        with open("Data/specie_encoding.pkl","rb") as f:
            self.species = pkl.load(f)
    
        try:
            shutil.rmtree("Data/Hierarchial Data")
            os.mkdir("Data/Hierarchial Data")
            
        except:
            os.mkdir("Data/Hierarchial Data")
            
        os.mkdir("Data/Hierarchial Data/train")
        os.mkdir("Data/Hierarchial Data/test")
        
        for cls in hd.keys():
            os.mkdir("Data/Hierarchial Data/train/"+str(cls))
            os.mkdir("Data/Hierarchial Data/test/"+str(cls))
            for order in hd[cls].keys():
                os.mkdir("Data/Hierarchial Data/train/"+str(cls)+"/"+str(order))
                os.mkdir("Data/Hierarchial Data/test/"+str(cls)+"/"+str(order))
                for fam in hd[cls][order].keys():
                    os.mkdir("Data/Hierarchial Data/train/"+str(cls)+"/"+str(order)+"/"+str(fam))
                    os.mkdir("Data/Hierarchial Data/test/"+str(cls)+"/"+str(order)+"/"+str(fam))
                    for gen in hd[cls][order][fam].keys():
                        os.mkdir("Data/Hierarchial Data/train/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen))
                        os.mkdir("Data/Hierarchial Data/test/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen))
                        for spe in hd[cls][order][fam][gen]:
                            try:
                                os.mkdir("Data/Hierarchial Data/train/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen)+"/"+str(spe))
                                os.mkdir("Data/Hierarchial Data/test/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen)+"/"+str(spe))
                            except:
                                pass

        

    def train_test_src_dest_map(self, train_size=0.85):
        df = pd.read_csv("occurences_train1.csv", low_memory=False)
        l = list(range(df.shape[0]))
        np.random.shuffle(l)
        split = int(train_size *len(l)) 
        for i in l[:split]:
            cls = df['class'][i]
            order = df['order'][i]
            fam = df['family'][i]
            gen = df['genus'][i]
            spe = df['species_glc_id'][i]
            self.paths.append(("patchTrain/"+str(df['patch_dirname'][i])+"/patch_"+str(df['patch_id'][i])+".tif",
                               "Data/Hierarchial Data/train/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen)+"/"+str(spe)))
        
        for i in l[split:]:
            cls = df['class'][i]
            order = df['order'][i]
            fam = df['family'][i]
            gen = df['genus'][i]
            spe = df['species_glc_id'][i]
            self.paths.append(("patchTrain/"+str(df['patch_dirname'][i])+"/patch_"+str(df['patch_id'][i])+".tif",
                               "Data/Hierarchial Data/test/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen)+"/"+str(spe)))    
    def parallelized_copying(self,item):
        global i
        i+=1
        if(i%100 == 0): 
            print(i, item[1])
        shutil.copy(item[0],item[1])
        
    def copy_data(self):
            with Pool(16) as p:
                #[p.apply_async(self.parallelized_copying, args=(x,)) for x in self.paths]
                p.map(self.parallelized_copying,self.paths)
                p.close()
                p.join()

In [8]:
ob = data_preprocessing()

In [9]:
ob.train_test_src_dest_map(train_size=0.7)

In [10]:
ob.copy_data()

100 Data/Hierarchial Data/train/Liliopsida/Poales/Poaceae/Brachypodium/271
100 Data/Hierarchial Data/train/Liliopsida/Poales/Juncaceae/Juncus/1579
100 Data/Hierarchial Data/train/Magnoliopsida/Asterales/Asteraceae/Lapsana/115
100 Data/Hierarchial Data/train/Liliopsida/Asparagales/Asparagaceae/Leopoldia/684
100 Data/Hierarchial Data/train/Magnoliopsida/Lamiales/Lamiaceae/Galeopsis/486
100 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Trigonella/3084
100 Data/Hierarchial Data/train/Magnoliopsida/Caryophyllales/Caryophyllaceae/Sagina/851
100 Data/Hierarchial Data/train/Magnoliopsida/Apiales/Apiaceae/Anthriscus/224
100 Data/Hierarchial Data/train/Magnoliopsida/Malpighiales/Linaceae/Linum/617
100 Data/Hierarchial Data/train/Liliopsida/Asparagales/Orchidaceae/Ophrys/716
100 Data/Hierarchial Data/train/Magnoliopsida/Lamiales/Plantaginaceae/Veronica/1001
100 Data/Hierarchial Data/train/Magnoliopsida/Gentianales/Rubiaceae/Galium/488
100 Data/Hierarchial Data/train/Magnoliopsida/Gen

700 Data/Hierarchial Data/train/Magnoliopsida/Brassicales/Brassicaceae/Sisymbrium/902
700 Data/Hierarchial Data/train/Liliopsida/Poales/Poaceae/Digitaria/2281
700 Data/Hierarchial Data/train/Magnoliopsida/Caryophyllales/Caryophyllaceae/Polycarpon/781
700 Data/Hierarchial Data/train/Magnoliopsida/Geraniales/Geraniaceae/Geranium/4
700 Data/Hierarchial Data/train/Magnoliopsida/Lamiales/Plantaginaceae/Plantago/772
800 Data/Hierarchial Data/train/Magnoliopsida/Malpighiales/Euphorbiaceae/Euphorbia/2354
700 Data/Hierarchial Data/train/Magnoliopsida/Ranunculales/Papaveraceae/Chelidonium/137
700 Data/Hierarchial Data/train/Magnoliopsida/Boraginales/Boraginaceae/Myosotis/689
800 Data/Hierarchial Data/train/Liliopsida/Poales/Poaceae/Dactylis/405
700 Data/Hierarchial Data/train/Magnoliopsida/Apiales/Apiaceae/Peucedanum/2417
800 Data/Hierarchial Data/train/Magnoliopsida/Lamiales/Plantaginaceae/Veronica/1001
800 Data/Hierarchial Data/train/Magnoliopsida/Asterales/Asteraceae/Crepis/386
800 Data/Hiera

1300 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Lotus/95
1300 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Hippocrepis/546
1400 Data/Hierarchial Data/train/Liliopsida/Poales/Poaceae/Poa/775
1400 Data/Hierarchial Data/train/Magnoliopsida/Rosales/Rosaceae/Prunus/1
1400 Data/Hierarchial Data/train/Liliopsida/Poales/Cyperaceae/Eleocharis/2330
1300 Data/Hierarchial Data/train/Magnoliopsida/Caryophyllales/Caryophyllaceae/Silene/2474
1400 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Trifolium/1207
1300 Data/Hierarchial Data/train/Liliopsida/Asparagales/Orchidaceae/Orchis/43
1400 Data/Hierarchial Data/train/Magnoliopsida/Brassicales/Brassicaceae/Raphanus/828
1300 Data/Hierarchial Data/train/Magnoliopsida/Apiales/Apiaceae/Oenanthe/705
1400 Data/Hierarchial Data/train/Magnoliopsida/Sapindales/Sapindaceae/Aesculus/182
1400 Data/Hierarchial Data/train/Magnoliopsida/Ranunculales/Papaveraceae/Chelidonium/137
1400 Data/Hierarchial Data/train/Magnoliopsida/L

1900 Data/Hierarchial Data/train/Magnoliopsida/Apiales/Araliaceae/Hedera/84
2000 Data/Hierarchial Data/train/Magnoliopsida/Asterales/Asteraceae/Lactuca/120
1900 Data/Hierarchial Data/train/Magnoliopsida/Brassicales/Brassicaceae/Pseudoturritis/806
2000 Data/Hierarchial Data/train/Magnoliopsida/Fagales/Betulaceae/Alnus/1304
2100 Data/Hierarchial Data/train/Liliopsida/Poales/Poaceae/Poa/776
2000 Data/Hierarchial Data/train/Magnoliopsida/Brassicales/Brassicaceae/Arabidopsis/230
2000 Data/Hierarchial Data/train/Magnoliopsida/Lamiales/Lamiaceae/Lamium/587
2100 Data/Hierarchial Data/train/Magnoliopsida/Gentianales/Gentianaceae/Centaurium/326
1900 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Trifolium/956
2100 Data/Hierarchial Data/train/Liliopsida/Poales/Poaceae/Rostraria/846
2000 Data/Hierarchial Data/train/Magnoliopsida/Lamiales/Plantaginaceae/Veronica/1001
2000 Data/Hierarchial Data/train/Magnoliopsida/Asterales/Asteraceae/Sonchus/912
2000 Data/Hierarchial Data/train/Magnolio

2600 Data/Hierarchial Data/train/Magnoliopsida/Apiales/Apiaceae/Laserpitium/1592
2700 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Trifolium/1206
2600 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Bituminaria/85
2500 Data/Hierarchial Data/train/Magnoliopsida/Asterales/Asteraceae/Picris/98
2700 Data/Hierarchial Data/train/Polypodiopsida/Polypodiales/Athyriaceae/Athyrium/1420
2600 Data/Hierarchial Data/train/Magnoliopsida/Apiales/Apiaceae/Pimpinella/2378
2700 Data/Hierarchial Data/train/Magnoliopsida/Dipsacales/Dipsacaceae/Scabiosa/3280
2700 Data/Hierarchial Data/train/Magnoliopsida/Asterales/Asteraceae/Filago/348
2600 Data/Hierarchial Data/train/Magnoliopsida/Gentianales/Apocynaceae/Cynanchum/2523
2600 Data/Hierarchial Data/train/Magnoliopsida/Rosales/Rosaceae/Fragaria/476
2700 Data/Hierarchial Data/train/Magnoliopsida/Ranunculales/Papaveraceae/Chelidonium/137
2700 Data/Hierarchial Data/train/Magnoliopsida/Fabales/Fabaceae/Trifolium/956
2700 Data/Hierarchial D

...

10800 Data/Hierarchial Data/test/Magnoliopsida/Malpighiales/Salicaceae/Populus/792
10800 Data/Hierarchial Data/test/Magnoliopsida/Ericales/Ericaceae/Erica/1783
11100 Data/Hierarchial Data/test/Magnoliopsida/Asterales/Asteraceae/Sonchus/912
11000 Data/Hierarchial Data/test/Liliopsida/Asparagales/Amaryllidaceae/Allium/18
10900 Data/Hierarchial Data/test/Magnoliopsida/Lamiales/Orobanchaceae/Rhinanthus/1089
10800 Data/Hierarchial Data/test/Magnoliopsida/Caryophyllales/Caryophyllaceae/Silene/118
10900 Data/Hierarchial Data/test/Magnoliopsida/Lamiales/Lamiaceae/Lavandula/1602
10900 Data/Hierarchial Data/test/Liliopsida/Poales/Poaceae/Melica/1660
10700 Data/Hierarchial Data/test/Liliopsida/Asparagales/Orchidaceae/Orchis/722
11100 Data/Hierarchial Data/test/Magnoliopsida/Rosales/Urticaceae/Parietaria/16
11000 Data/Hierarchial Data/test/Magnoliopsida/Lamiales/Plantaginaceae/Cymbalaria/156
11000 Data/Hierarchial Data/test/Magnoliopsida/Asterales/Asteraceae/Bellis/48
11000 Data/Hierarchial Data/t

11600 Data/Hierarchial Data/test/Liliopsida/Poales/Poaceae/Bromus/221
11600 Data/Hierarchial Data/test/Magnoliopsida/Asterales/Asteraceae/Filago/348
11700 Data/Hierarchial Data/test/Magnoliopsida/Lamiales/Lamiaceae/Scutellaria/882
11600 Data/Hierarchial Data/test/Magnoliopsida/Ranunculales/Ranunculaceae/Ficaria/473
11400 Data/Hierarchial Data/test/Magnoliopsida/Solanales/Convolvulaceae/Convolvulus/161
11500 Data/Hierarchial Data/test/Magnoliopsida/Saxifragales/Saxifragaceae/Saxifraga/863
11500 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Trifolium/956
11500 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Robinia/841
11800 Data/Hierarchial Data/test/Magnoliopsida/Asterales/Asteraceae/Erigeron/455
11600 Data/Hierarchial Data/test/Magnoliopsida/Brassicales/Brassicaceae/Sinapis/901
11700 Data/Hierarchial Data/test/Magnoliopsida/Dipsacales/Adoxaceae/Viburnum/1252
11400 Data/Hierarchial Data/test/Pinopsida/Pinales/Pinaceae/Pinus/1034
11700 Data/Hierarchial Data/test/Ma

12300 Data/Hierarchial Data/test/Liliopsida/Asparagales/Amaryllidaceae/Allium/200
12400 Data/Hierarchial Data/test/Magnoliopsida/Lamiales/Plantaginaceae/Linaria/616
12100 Data/Hierarchial Data/test/Magnoliopsida/Apiales/Apiaceae/Sison/2479
12300 Data/Hierarchial Data/test/Magnoliopsida/Malvales/Malvaceae/Tilia/2453
12000 Data/Hierarchial Data/test/Magnoliopsida/Asterales/Asteraceae/Erigeron/2311
12100 Data/Hierarchial Data/test/Magnoliopsida/Malvales/Cistaceae/Cistus/11
12100 Data/Hierarchial Data/test/Magnoliopsida/Ranunculales/Ranunculaceae/Ficaria/473
12200 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Cytisus/125
12300 Data/Hierarchial Data/test/Magnoliopsida/Lamiales/Lamiaceae/Lamium/585
12300 Data/Hierarchial Data/test/Magnoliopsida/Ranunculales/Berberidaceae/Berberis/1435
12300 Data/Hierarchial Data/test/Magnoliopsida/Gentianales/Gentianaceae/Gentiana/2322
12200 Data/Hierarchial Data/test/Magnoliopsida/Rosales/Rosaceae/Filipendula/475
12200 Data/Hierarchial Data/test

12900 Data/Hierarchial Data/test/Magnoliopsida/Asterales/Asteraceae/Sonchus/119
12600 Data/Hierarchial Data/test/Liliopsida/Asparagales/Asparagaceae/Ornithogalum/124
12900 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Vicia/1015
12900 Data/Hierarchial Data/test/Magnoliopsida/Malpighiales/Hypericaceae/Hypericum/2153
12500 Data/Hierarchial Data/test/Liliopsida/Asparagales/Orchidaceae/Anacamptis/31
12700 Data/Hierarchial Data/test/Magnoliopsida/Apiales/Apiaceae/Falcaria/1461
12800 Data/Hierarchial Data/test/Magnoliopsida/Caryophyllales/Caryophyllaceae/Cerastium/333
12800 Data/Hierarchial Data/test/Liliopsida/Asparagales/Orchidaceae/Himantoglossum/162
12800 Data/Hierarchial Data/test/Magnoliopsida/Cucurbitales/Cucurbitaceae/Bryonia/153
12800 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Trifolium/1224
12900 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Robinia/841
12800 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Hippocrepis/28
13100 Data/H

13400 Data/Hierarchial Data/test/Magnoliopsida/Dipsacales/Caprifoliaceae/Centranthus/90
13300 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Astragalus/2809
13400 Data/Hierarchial Data/test/Magnoliopsida/Rosales/Rosaceae/Fragaria/476
13600 Data/Hierarchial Data/test/Magnoliopsida/Asterales/Asteraceae/Erigeron/368
13400 Data/Hierarchial Data/test/Magnoliopsida/Oxalidales/Oxalidaceae/Oxalis/734
13500 Data/Hierarchial Data/test/Magnoliopsida/Gentianales/Rubiaceae/Cruciata/390
13400 Data/Hierarchial Data/test/Magnoliopsida/Ranunculales/Papaveraceae/Papaver/64
13600 Data/Hierarchial Data/test/Magnoliopsida/Brassicales/Brassicaceae/Biscutella/267
13400 Data/Hierarchial Data/test/Magnoliopsida/Caryophyllales/Caryophyllaceae/Cerastium/332
13600 Data/Hierarchial Data/test/Liliopsida/Poales/Poaceae/Digitaria/2281
13400 Data/Hierarchial Data/test/Magnoliopsida/Lamiales/Lamiaceae/Lamium/12
13600 Data/Hierarchial Data/test/Magnoliopsida/Fabales/Fabaceae/Trifolium/950
13600 Data/Hierarchi